In [70]:
import fastf1 as f1 # f1 data library
from fastf1.ergast import Ergast
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.io import show
from collections import OrderedDict

In [53]:
event_schedule = f1.get_event_schedule(2024)

finished_events = event_schedule[event_schedule.Session5DateUtc < np.datetime64(pd.Timestamp('today', tz='UTC'))]

num_finished_events = len(finished_events)

finished_events

C:\Users\kevin.sweitzer\AppData\Local\Temp\ipykernel_22392\2196660640.py:3: DeprecationWarning:

parsing timezone aware datetimes is deprecated; this will raise an error in the future



,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,...,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True
2,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2024,2024-03-09,Saudi Arabian Grand Prix,conventional,Practice 1,2024-03-07 16:30:00+03:00,2024-03-07 13:30:00,...,Practice 3,2024-03-08 16:30:00+03:00,2024-03-08 13:30:00,Qualifying,2024-03-08 20:00:00+03:00,2024-03-08 17:00:00,Race,2024-03-09 20:00:00+03:00,2024-03-09 17:00:00,True
3,3,Australia,Melbourne,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2024,2024-03-24,Australian Grand Prix,conventional,Practice 1,2024-03-22 12:30:00+11:00,2024-03-22 01:30:00,...,Practice 3,2024-03-23 12:30:00+11:00,2024-03-23 01:30:00,Qualifying,2024-03-23 16:00:00+11:00,2024-03-23 05:00:00,Race,2024-03-24 15:00:00+11:00,2024-03-24 04:00:00,True
4,4,Japan,Suzuka,FORMULA 1 MSC CRUISES JAPANESE GRAND PRIX 2024,2024-04-07,Japanese Grand Prix,conventional,Practice 1,2024-04-05 11:30:00+09:00,2024-04-05 02:30:00,...,Practice 3,2024-04-06 11:30:00+09:00,2024-04-06 02:30:00,Qualifying,2024-04-06 15:00:00+09:00,2024-04-06 06:00:00,Race,2024-04-07 14:00:00+09:00,2024-04-07 05:00:00,True
5,5,China,Shanghai,FORMULA 1 LENOVO CHINESE GRAND PRIX 2024,2024-04-21,Chinese Grand Prix,sprint_qualifying,Practice 1,2024-04-19 11:30:00+08:00,2024-04-19 03:30:00,...,Sprint,2024-04-20 11:00:00+08:00,2024-04-20 03:00:00,Qualifying,2024-04-20 15:00:00+08:00,2024-04-20 07:00:00,Race,2024-04-21 15:00:00+08:00,2024-04-21 07:00:00,True
6,6,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2024,2024-05-05,Miami Grand Prix,sprint_qualifying,Practice 1,2024-05-03 12:30:00-04:00,2024-05-03 16:30:00,...,Sprint,2024-05-04 12:00:00-04:00,2024-05-04 16:00:00,Qualifying,2024-05-04 16:00:00-04:00,2024-05-04 20:00:00,Race,2024-05-05 16:00:00-04:00,2024-05-05 20:00:00,True
7,7,Italy,Imola,FORMULA 1 MSC CRUISES GRAN PREMIO DEL MADE IN ...,2024-05-19,Emilia Romagna Grand Prix,conventional,Practice 1,2024-05-17 13:30:00+02:00,2024-05-17 11:30:00,...,Practice 3,2024-05-18 12:30:00+02:00,2024-05-18 10:30:00,Qualifying,2024-05-18 16:00:00+02:00,2024-05-18 14:00:00,Race,2024-05-19 15:00:00+02:00,2024-05-19 13:00:00,True
8,8,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2024,2024-05-26,Monaco Grand Prix,conventional,Practice 1,2024-05-24 13:30:00+02:00,2024-05-24 11:30:00,...,Practice 3,2024-05-25 12:30:00+02:00,2024-05-25 10:30:00,Qualifying,2024-05-25 16:00:00+02:00,2024-05-25 14:00:00,Race,2024-05-26 15:00:00+02:00,2024-05-26 13:00:00,True
9,9,Canada,Montréal,FORMULA 1 AWS GRAND PRIX DU CANADA 2024,2024-06-09,Canadian Grand Prix,conventional,Practice 1,2024-06-07 13:30:00-04:00,2024-06-07 17:30:00,...,Practice 3,2024-06-08 12:30:00-04:00,2024-06-08 16:30:00,Qualifying,2024-06-08 16:00:00-04:00,2024-06-08 20:00:00,Race,2024-06-09 14:00:00-04:00,2024-06-09 18:00:00,True
10,10,Spain,Barcelona,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2024,2024-06-23,Spanish Grand Prix,conventional,Practice 1,2024-06-21 13:30:00+02:00,2024-06-21 11:30:00,...,Practice 3,2024-06-22 12:30:00+02:00,2024-06-22 10:30:00,Qualifying,2024-06-22 16:00:00+02:00,2024-06-22 14:00:00,Race,2024-06-23 15:00:00+02:00,2024-06-23 13:00:00,True


In [43]:
# The initial creation of a nb that has season long data tracking

# TODO: Drivers/Constructors Points graphs, Quali Performance, Driver standings

ergast = Ergast()
races = ergast.get_race_schedule(2024)
results = []

for rnd, race in races['raceName'].items():
    if rnd+1 not in finished_events['RoundNumber']:
        break
    curr_race_results = ergast.get_race_results(season=2024, round=rnd+1)
    curr_race_results = curr_race_results.content[0]

    # If there is a sprint, get the results as well
    curr_sprint_results = ergast.get_sprint_results(season=2024, round=rnd + 1)
    if curr_sprint_results.content and curr_sprint_results.description['round'][0] == rnd + 1:
        curr_race_results = pd.merge(curr_race_results, curr_sprint_results.content[0], on='driverCode', how='left')
        # Add sprint points and race points to get the total
        curr_race_results['points'] = curr_race_results['points_x'] + curr_race_results['points_y']
        curr_race_results.drop(columns=['points_x', 'points_y'], inplace=True)

    # Add round no. and grand prix name
    curr_race_results['round'] = rnd + 1
    curr_race_results['race'] = race.removesuffix(' Grand Prix')
    curr_results = curr_race_results[['round', 'race', 'driverCode', 'points']]  # Keep useful cols.
    results.append(curr_race_results)

results = pd.concat(results)
races = results['race'].drop_duplicates()
results = results.pivot(index='driverCode', columns='round', values='points')

results

round,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
driverCode,,,,,,,,,,,,,,,,,
ALB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,6.0
ALO,2.0,10.0,4.0,8.0,7.0,2.0,0.0,0.0,8.0,0.0,0.0,4.0,0.0,4.0,1.0,0.0,8.0
BEA,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
BOT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.0
GAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0
HAM,6.0,2.0,0.0,2.0,9.0,8.0,8.0,7.0,13.0,15.0,15.0,25.0,15.0,25.0,4.0,10.0,2.0
HUL,0.0,1.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0
LEC,12.0,16.0,19.0,12.0,17.0,22.0,15.0,25.0,0.0,10.0,2.0,0.0,12.0,15.0,15.0,25.0,18.0


In [ ]:
temp = "bruh"

In [91]:
def sort_func(e):
    return e[-1][-1]

cummulative_driver_race_results = {}
for race in results:
    for driver, pts in results[race].items():
        if driver in cummulative_driver_race_results:
            curr_driver_total_pts = cummulative_driver_race_results[driver][-1]
            cummulative_driver_race_results[driver].append(0 + curr_driver_total_pts if pd.isnull(pts) else pts + curr_driver_total_pts)
        else:
            cummulative_driver_race_results[driver] = [0 if pd.isnull(pts) else pts]
old_keys = list(cummulative_driver_race_results.keys())
for driver in old_keys:
    new_key = f"{driver} ({int(cummulative_driver_race_results[driver][-1])} pts)"
    cummulative_driver_race_results[new_key] = cummulative_driver_race_results.pop(driver)
            
temp = list(cummulative_driver_race_results.items())
temp.sort(key=sort_func, reverse=True)
cummulative_driver_race_results = OrderedDict(temp)

In [104]:
fig = px.line(
    cummulative_driver_race_results,
    labels={'index': 'Round #',
            'value': 'Championship Points',
            'variable': 'Driver'},
    markers=True
)

show(fig)

In [103]:
curr_race_results = ergast.get_constructor_standings(season=2024, round=17)

curr_race_results.content[0]

,position,positionText,points,wins,constructorId,constructorUrl,constructorName,constructorNationality
0,1,1,476.0,4,mclaren,http://en.wikipedia.org/wiki/McLaren,McLaren,British
1,2,2,456.0,7,red_bull,http://en.wikipedia.org/wiki/Red_Bull_Racing,Red Bull,Austrian
2,3,3,425.0,3,ferrari,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Ferrari,Italian
3,4,4,309.0,3,mercedes,http://en.wikipedia.org/wiki/Mercedes-Benz_in_...,Mercedes,German
4,5,5,82.0,0,aston_martin,http://en.wikipedia.org/wiki/Aston_Martin_in_F...,Aston Martin,British
5,6,6,34.0,0,rb,http://en.wikipedia.org/wiki/RB_Formula_One_Team,RB F1 Team,Italian
6,7,7,29.0,0,haas,http://en.wikipedia.org/wiki/Haas_F1_Team,Haas F1 Team,American
7,8,8,16.0,0,williams,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,Williams,British
8,9,9,13.0,0,alpine,http://en.wikipedia.org/wiki/Alpine_F1_Team,Alpine F1 Team,French
9,10,10,0.0,0,sauber,http://en.wikipedia.org/wiki/Sauber_Motorsport,Sauber,Swiss
